Alperen Akgül 233405014

In [ ]:
# Gerekli kütüphanelerin eklenmesi
import pandas as pd
import string
from string import printable
import re
import random
import pickle

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

In [ ]:
# tweets.csv dosyasındaki tweetlerin okunması.
dframe = pd.read_csv("/content/tweets.csv", encoding='unicode_escape', on_bad_lines='skip', sep=';', quoting = 3)
dframe

,text,emotion
0,"""""why is this guy talking about oppenheimer, j...",-1.0
1,no i meant like barbie as in the barbie movies...,-1.0
2,Oppenheimer was a good movie but I feel like t...,1.0
3,Finally watching Oppenheimer. And JESUS the ...,-1.0
4,"""""ONE WEEK till the 29th Annual Critics Choice...",-1.0
...,...,...
2315,"""""i watched oppenheimer today. lips were movin...",1.0
2316,i watched oppenheimer yesterday and i was smil...,-1.0
2317,"""""watched oppenheimer and as someone who's not...",-1.0
2318,watched oppenheimer with my whole family then ...,-1.0


In [ ]:
# emotion sütunundaki değeri 1 veya 0 olan satırları diğer satırlardan ayırıyoruz.
# 1 'Olumlu' anlamına gelirken, 0 'Olumsuz' anlamına gelmekte.
# Bu sayede DataFrame deki duygu belirtmeyen satırlardan kurtulmuş oluyoruz.
dframe = dframe.loc[dframe["emotion"].isin([1.0, 0.0])]

In [ ]:
# DataFrame de 589 tane olumlu yorum bulunurken, 136 tane olumsuz yorum bulunmakta.
print("1: Pozitif", "0: Negatif\n")
dframe["emotion"].value_counts()

1: Pozitif 0: Negatif



1.0    589
0.0    136
Name: emotion, dtype: int64

In [ ]:
# Metinden çıkaracağımız dolgu sözcükleri
# kaynak: https://gist.github.com/sebleier/554280
stop_words = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]

In [ ]:
# Metinleri noktalama işaretlerinden, dolgu sözcüklerinden temizleyen fonksiyon.
def clean_text(text):
    text = " ".join(text.split())
    text = text.lower()
    text = text.replace("\\n", " ")
    text = re.sub("[0-9]+", "", text)
    text = re.sub("%|(|)|-", "", text)
    text = text.translate(str.maketrans("", "", string.punctuation))
    text = [t for t in text.split() if t not in stop_words]
    return " ".join(text)


In [ ]:
# Metinleri clean_text fonksiyonu kullanarak temizliyoruz.
# Temizlenen metini clean sütununa ekliyoruz.
# İlk satır sütun başlıkları olduğu için indeksi 1 den başlattık.
dframe["clean"] = dframe.apply(lambda row: clean_text(row["text"]), axis=1)

<ipython-input-105-c200ae27c0b6>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dframe["clean"] = dframe.apply(lambda row: clean_text(row["text"]), axis=1)


In [ ]:
# Metinin içinde hala özel karakterler bulunmakta.
# Bu fonksiyon sayesinde sadece harflerden ve rakamlardan oluşan bir metin elde ediyoruz.
# Elde edilen metin clean sütunundaki metin ile değiştiriliyor.
def getOnlyCharacters(texts):
    _type = None
    result = ''

    if type(texts).__name__ == 'bytes':
        _type = 'bytes'
        texts = texts.decode('utf-16-le','ignore')
    else:
        _type = 'str'
        texts = bytes(texts, 'utf-16-le').decode('utf-8', 'ignore')

    texts = str(texts)
    for text in texts:
        if text in printable:
            result += text

    if _type == 'bytes':
        result = result.encode('utf-16-le')

    return result

In [ ]:
# getOnlyCharacters fonksiyonunu ilk satır hariç her satır için çağırıyoruz.
dframe["clean"] = dframe.apply(lambda row: getOnlyCharacters(row["clean"]), axis=1)

<ipython-input-107-00e23f1172ee>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dframe["clean"] = dframe.apply(lambda row: getOnlyCharacters(row["clean"]), axis=1)


In [ ]:
# Şu anda elimizdeki DataFrame bu şekilde gözüküyor.
# text ve clean sütunundaki metinin ne kadar farklı olduğu görülebilir.
dframe.head()

,text,emotion,clean
2,Oppenheimer was a good movie but I feel like t...,1.0,oppenheimer good movie feel like way jumps aro...
6,Oppenheimer more like this film is so good I r...,1.0,oppenheimer like film good really enjoyed watc...
8,"""""REWATCH-#Oppenheimer. My 11yo sonÃ¢â¬â¢s r...",1.0,rewatchoppenheimer yo sons review its one nola...
11,"""""Finally got around to watching Oppenheimer a...",1.0,finally got around watching oppenheimer goddam...
20,Finally watched Oppenheimer and itÃ¢â¬â¢s go...,1.0,finally watched oppenheimer its good like ever...


In [ ]:
# Bu adımdan sonra yapacağımız işler için clean ve emotion satırını numpy dizisine dönüştürüyoruz.
# X dizisi temizlenmiş metinleri tutarken,
# Y dizisi 1 ve 0 etiketlerini tutmakta.
X = dframe["clean"].to_numpy()
y = dframe["emotion"].to_numpy()

In [ ]:
# Toplam 725 tane olumlu veya olumsuz metinimiz bulunmakta.
print("X:", len(X), "Y:", len(y))

X: 725 Y: 725


In [ ]:
# X ve Y dizilerini %20 si test verisi olacak şekilde ayırıyoruz.
# Ayrıca verilerin karışık şekilde dağılması için random_state değeri atadık.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=23)

In [ ]:
# X eğitim ve X test verilerini TF-IDF kullanarak rakamsal değerler haline getiriyoruz.
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
# İlk olarak Karar Ağaçlarını kullanacağız.
# DecTree.fit() fonksiyonu ile modeli eğitiyoruz.
# DecTree.score kullanarak model, ayırdığımız %20'lik kısmın etiketlerini tahmin etmeye çalışıyor.
# Tahminden sonra doğruluk oranını döndürüyor.
# Yaklaşık %88 doğruluk oranına sahibiz.
DecTree = DecisionTreeClassifier()
DecTree.fit(X_train, y_train)
print("Karar Ağacı Skoru: ", DecTree.score(X_test, y_test))

Karar Ağacı Skoru:  0.8827586206896552


In [ ]:
# Şimdi aynı işlemi Naive Bayes algoritması için tekrarlıyoruz.
# Bu sefer yaklaşık %86 doğruluk oranı elde ediyoruz.
NB = MultinomialNB()
NB.fit(X_train, y_train)
print("Naive Bayes Skoru: " , NB.score(X_test, y_test))

Naive Bayes Skoru:  0.8620689655172413


In [ ]:
# Aynı işlemi Support Vector Machine için tekrarlıyoruz.
# Parametre olarak linear seçiyoruz.
# Yaklaşık %93 doğruluk oranı elde ediyoruz.
SVM = SVC(kernel="linear")
SVM.fit(X_train, y_train)
print("Support Vector Machine Skoru: ", SVM.score(X_test, y_test))

Support Vector Machine Skoru:  0.9379310344827586


In [ ]:
# Modeli diske kaydedebiliriz. Örneğin Karar ağacı için:
with open("tree.pkl", "wb") as f:
    pickle.dump(DecTree, f)

# Aynı şekilde vektörleştiriciyi de diske kaydedebiliriz.
with open("vectorizer.pkl", "wb") as f:
    pickle.dump(vectorizer, f)

In [ ]:
# Modeli diskten tekrar yüklemek.
clf_loaded = None
with open("tree.pkl", "rb") as f:
    clf_loaded = pickle.load(f)

# Vektörleştiriciyi diskten geri yüklemek.
vectorizer_loaded = None
with open("vectorizer.pkl", "rb") as f:
    vectorizer_loaded = pickle.load(f)

In [ ]:
# Şimdi örnek olarak 4 tane cümle veriyoruz.
# Cümlelerin temizlenmiş halleri çıktıda görülebilir.
# Karar Ağacı modelinin tahmin etmesini istiyoruz.
# Yapılan tahminler ise doğrudur.
examples = [
    "it was very good movie",
    "it was very bad movie",
    "it was boring",
    "oppenheimer was awesome"
]

examples = [clean_text(e) for e in examples]
print("Temizlenmiş Metinler:\n", examples)
examples_vec = vectorizer_loaded.transform(examples)
clf_loaded.predict(examples_vec)

Temizlenmiş Metinler:
 ['good movie', 'bad movie', 'boring', 'oppenheimer awesome']


array([1., 0., 0., 1.])